In [1]:
import gym
import ray

In [2]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [3]:
# Initialize Ray
ray.shutdown()
ray.init(ignore_reinit_error=True)

2020-09-27 16:02:45,062	INFO resource_spec.py:223 -- Starting Ray with 35.89 GiB memory available for workers and up to 17.97 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-27 16:02:45,486	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.65',
 'raylet_ip_address': '192.168.1.65',
 'redis_address': '192.168.1.65:6379',
 'object_store_address': '/tmp/ray/session_2020-09-27_16-02-45_061032_20740/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-27_16-02-45_061032_20740/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-09-27_16-02-45_061032_20740'}

In [5]:
print("Dashboard URL: http://{}".format(ray.get_webui_url()))

Dashboard URL: http://localhost:8265


### Register MineRL Gym Environment to RLlib

In [6]:
import minerl
from gym import envs

/home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
from minerl_rllib.envs.env import MineRLRandomDebugEnv

In [8]:
# Register MineRL Gym Environment to RLLIB
# https://docs.ray.io/en/latest/rllib-env.html
from ray.tune.registry import register_env

def minerl_env_creator(env_config):
    import minerl
    
    if 'minerl_env_name' in env_config:
        # TODO use logger
        print('MineRL Env Name found...')
        env_name = env_config['minerl_env_name']
    else:
        # TODO use logger
        print('No MineRL Env name specified, using MineRLNavigateDense-v0')
        env_name = 'MineRLNavigateDense-v0'
        
        
#     # Check minerl environments are imported
#     # This only checks Gym environment not the ones registered in Ray!
#     all_envs = envs.registry.all()
#     env_ids = [env_spec.id for env_spec in all_envs]
#     print(env_ids)

# This doesnt work, need wrapper to discretize the action space for DQN
#     minerl_env = gym.make(env_name) 
    
    # Second version but has .nan rewards
#     core_env = gym.make(env_name) # A MineRLNavigate-v0 env
#     minerl_env = wrap_env(core_env, test=False)
    
#     minerl_env = gym.make('MineRLNavigateDense-v0')
    
    return minerl_env  

register_env("minerl", minerl_env_creator)

In [9]:
from minerl_rllib.envs import register

In [10]:
register()

In [11]:
obfuscated_envs = minerl.herobraine.envs.obfuscated_envs

In [12]:
obfuscated_envs[0].ac_enc

<function minerl.herobraine.wrappers.obfuscation_wrapper.Obfuscated._get_obfuscator.<locals>.make_func.<locals>.func(x)>

In [15]:
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.dqn import DQNTrainer

tune.run("PPO",
         config={"env": "MineRLNavigateDenseVectorObf-v0",
                 "monitor":False, 
                 "num_gpus": 1,
                 "num_workers": 1,
                 "num_envs_per_worker":1,
                 "eager":False,
                 "use_pytorch":True,
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 },
         stop={"training_iteration": 10})  
# Config notes:
# "log_level": "INFO" for verbose,
# "eager": True for eager execution,

Trial name,status,loc
PPO_MineRLNavigateDenseVectorObf-v0_29055_00000,RUNNING,


(pid=20860) WARNING:tensorflow:From /home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=20860) Instructions for updating:
(pid=20860) non-resource variables are not supported in the long term
(pid=20860) /home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=20860)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=20860) 2020-09-27 16:07:08,204	WARNING deprecation.py:29 -- DeprecationWarning: `use_pytorch` has been deprecated. Use `framework=torch` instead. This will raise an error in the future!
(pid=20860) 2020-09-27 16:07:08,204	WARNING deprecation.py:29 -- DeprecationWarning: `eager` has been deprecated. Use `framework=tfe` instead. This will raise an error in the future!
(pid=20860) 202

Result for PPO_MineRLNavigateDenseVectorObf-v0_29055_00000:
  custom_metrics: {}
  date: 2020-09-27_16-08-55
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: cd52130b4cb14f188950a86134d66947
  experiment_tag: '0'
  hostname: kokkgoblin
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.2
        cur_lr: 5.0e-05
        entropy: 90.75303053855896
        entropy_coeff: 0.0
        kl: 0.022007984633091837
        policy_loss: -0.02486593060893938
        total_loss: 0.017681708675809205
        vf_explained_var: 0.00016084685921669006
        vf_loss: 0.03814604436047375
    num_steps_sampled: 4000
    num_steps_trained: 4000
  iterations_since_restore: 1
  node_ip: 192.168.1.65
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 40.473825503355705
    ram_util_percent: 26.7308724

Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_29055_00000,RUNNING,192.168.1.65:20860,1,104.168,4000,nan


(pid=20874) WARNING:tensorflow:From /home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/ray/rllib/policy/tf_policy.py:869: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=20874) Instructions for updating:
(pid=20874) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=20877) WARNING:tensorflow:From /home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/ray/rllib/policy/tf_policy.py:869: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=20877) Instructions for updating:
(pid=20877) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=20870) /home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/minerl/herobraine/wrappers/obfuscation_wrapper.py:67: RuntimeWarning: invalid value encountered in true_divide
(pid=20870)   x[..., a:b] = e_x / e_x.sum(axis=-1)


...

Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_29055_00000,RUNNING,192.168.1.65:20860,9,622.768,36000,0.0124683


Result for PPO_MineRLNavigateDenseVectorObf-v0_29055_00000:
  custom_metrics: {}
  date: 2020-09-27_16-18-25
  done: true
  episode_len_mean: 6000.0
  episode_reward_max: 3.2809486389160156
  episode_reward_mean: 0.012468338012695312
  episode_reward_min: -3.4253692626953125
  episodes_this_iter: 0
  episodes_total: 6
  experiment_id: cd52130b4cb14f188950a86134d66947
  experiment_tag: '0'
  hostname: kokkgoblin
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 1.5187500000000003
        cur_lr: 5.0e-05
        entropy: 90.50423455238342
        entropy_coeff: 0.0
        kl: 0.013474712934112176
        policy_loss: -0.04066264955326915
        total_loss: 0.22766426880843937
        vf_explained_var: 7.715635001659393e-05
        vf_loss: 0.24786219839006662
    num_steps_sampled: 40000
    num_steps_trained: 40000
  iterations_since_restore: 10
  node_ip: 192.168.1.65
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_uti

Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_29055_00000,TERMINATED,,10,673.592,40000,0.0124683


Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_29055_00000,TERMINATED,,10,673.592,40000,0.0124683


(pid=20870) *** Aborted at 1601248708 (unix time) try "date -d @1601248708" if you are using GNU date ***
(pid=20870) PC: @                0x0 (unknown)
(pid=20870) *** SIGSEGV (@0x7fe993fff9d0) received by PID 22551 (TID 0x7fe9b6ea8740) from PID 18446744071897610704; stack trace: ***
(pid=20870)     @     0x7fe9b6a998a0 (unknown)
(pid=20870)     @     0x7fe9b6a8fbd8 __GI___pthread_timedjoin_ex
(pid=20870)     @     0x7fe9b487a2d3 std::thread::join()
(pid=20870)     @     0x7fe9b4da43a3 ray::gcs::GlobalStateAccessor::Disconnect()
(pid=20870)     @     0x7fe9b4c4918c __pyx_pw_3ray_7_raylet_19GlobalStateAccessor_5disconnect()
(pid=20870)     @     0x562bbc240e6a method_vectorcall_NOARGS
(pid=20870)     @     0x562bbc1d175e _PyEval_EvalFrameDefault.cold.2790
(pid=20870)     @     0x562bbc25c86b _PyFunction_Vectorcall.localalias.355
(pid=20870)     @     0x562bbc1d175e _PyEval_EvalFrameDefault.cold.2790
(pid=20870)     @     0x562bbc25ba92 _PyEval_EvalCodeWithName
(pid=20870)     @     0x5

In [11]:
import minerl
import gym
env = gym.make('MineRLNavigateDense-v0')


obs = env.reset()

done = False
while not done:
    action = env.action_space.sample() 
 
    # One can also take a no_op action with
    # action =env.action_space.noop()
    
 
    obs, reward, done, info = env.step(
        action)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-68980fa6ca4b>", line 16, in <module>
    obs, reward, done, info = env.step(
  File "/home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/gym/wrappers/time_limit.py", line 16, in step
    observation, reward, done, info = self.env.step(action)
  File "/home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/minerl/env/core.py", line 623, in step
    obs = comms.recv_message(self.client_socket)
  File "/home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/minerl/env/comms.py", line 62, in recv_message
    lengthbuf = recvall(sock, 4)
  File "/home/shyam/anaconda3/envs/py38/lib/python3.8/site-packages/minerl/env/comms.py", line 72, in recvall
    newbuf = sock.recv(count)
KeyboardInterrupt

During handling of the above exception, anoth

TypeError: object of type 'NoneType' has no len()